In [9]:
from IPython.display import display, HTML, Image, clear_output
import time
import requests
from bs4 import BeautifulSoup
import json
from ipywidgets import Output
from time import sleep
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

grid_1 = Image("memory_1.png", width=400) #stating the size of displayed grid
grid_2 = Image("memory_2.png", width=400)
grid_3 = Image("memory_3.png", width=400)
q5_level_1= Image("q5_level_1.png", width=400) #questions asked in relation with this grid named q5_level_1 and q5
q5= Image("q5.png", width=400)

In [10]:
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok
  

In [11]:
event_info = {
'type': '',
'description': '',
'time': -1
}

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    print(f"Button clicked: {btn.description}")
    
def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):
    
    start_wait = time.time()
    
    n_proc = int(max_rate*interval)+1

    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            
            time.sleep(interval)

    return event_info

#yes button function
def on_button_click():
    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSffAojpYUbeyM_PHH4AXIeOLKCD-Gt-ik-ASwsLO7SETvlQkQ/viewform"
    
    data_dict = {
        'name': user,
        'gender': gender, 
        'age': age,
        'medication': medication,
        'sleep': sleep,
        'stress': stress,
        'Level 1 score': total_score_1,
        'Level 2 score': total_score_2,
        'Level 3 score': total_score_3,
        'total_score': total_score,
        'time_taken': total_time_taken}
    
    print("Thanks - your data will be uploaded.")
    send_to_google_form(data_dict, form_url)

#no button function
def register_event():
    print("No problem, we hope you enjoyed the test.")

In [13]:
def run_memory_test ():
    question_bank_2=["What color was the heart?", "How many blank grids were there? (Enter your answer in words)", "Which shape has yellow color?", "What color was circle?", "What shape was in the indicated position?"]
    answer_2=["red","three","star", "blue", "rectangle"]

    question_bank_1=["What color was the square?", "How many shapes have the same color? (Enter your answer in words)", "What color was the 'plus' sign?", "What shape was between the triangle and diamond?",
                  "What shape is in the indicated position?"]
    answer_1=["green","two","pink", "circle", "pentagon"]
    
    question_bank_3=["What color was the heart?","What shape was inside the 'plus' sign?", "Which direction was the arrow in the green square pointing at? (left, right)","How many shapes were inside the star? (Enter your answer in words)",
                 "What color of star is in the indicated position?"]
    answer_3=["black","circle","right", "two", "yellow"]
    
    html_content = HTML('<p style="font-size: 25px;">Memory Test</p>')
    display(html_content) #title of the test

    global user #storing user as a global variable so that it can be stored in the form. 
    print("To generate an anonymous 4-letter unique user identifier please enter:")

    print ("")
    
    print("- two letters based on the initials (first and last name) of a childhood friend")
    print("- two letters based on the initials (first and last name) of a favourite actor / actress")

    print ("")
    
    print("e.g. if your friend was called Charlie Brown and film star was Tom Cruise")

    print("then your unique identifer would be CBTC")

    print("")
    
    user = input("Enter your anonymous ID: ")
    
    global gender
    gender = input("What is your gender? (Female or Male)")
    
    global age
    age = input("What is your age?")

    clear_output (wait=False)
    
    print(f"Hi {user}, welcome to memory test!!")
    print("")
    print("You will be given time to memorise the following image amd answer questions related to the image.")
    print("There are a total of three levels in this test.")
    print("")
    print("Before we could proceed to the test, here are some questions related to this test that you need to answer.")
    print("")
    
    #extra questions asked to allow better data analysis
    global medication, sleep, stress
    medication = input ("Are you currently taking any medications? (If yes, please specify the type of medication you are taking)")
    sleep = input ("On average, how many hours do you sleep in a day?")
    stress = input ("Are you currently experiencing high levels of stress or anxiety?")
    clear_output(wait=False)
    
    print("Thank you for answering the questions, you will now be directed to the test.")
    print("Good luck!")

    clear_output(wait=False)

    html_content_1 = HTML('<p style="font-size: 20px;">Level 1</p>') #displaying the title with desired font size
    
    #create output widget
    out1 = Output()
    out2 = Output() 

    #display output
    display(out1)
    display(out2)
     
    with out1: display(html_content_1, grid_2)

    #displaying time remaining and numbers in a loop
    for i in range(10, -1, -1): #countdown from 10 to 0
        out2.clear_output(wait=True)
        with out2: display(HTML(f"<h3>Time remaining: {i} seconds</h3>"))
            
        time.sleep(1)
        
    clear_output(wait=False)
    
    start_time_1 = time.time() #record current time
    
    answer = ""

    global total_score_1
    total_score_1 = 0 #initialise score to zero
    
    len_ques = len(question_bank_1) #counting the number of items in the list
    
    for i in range(len_ques):
        if i == 4:
            display(q5_level_1) #only display when condition met 
            
        ques = question_bank_2[i] #assign the corresponding question bank to the variable 'ques' from the list based on the current indec 'i'
        ans = answer_2[i] #same goes to 'ans'
        answer=input(ques) #request answers from user and store it in the variable 'answer'
        
        if answer == ans:
            print ("Correct!")
            total_score_1 = total_score_1 + 1
        else:
            print(f"Sorry, the correct answer is: {ans}") #disply the correct answer

        time.sleep(1)
        clear_output(wait=False)
            
    end_time_1=time.time() 
    time_taken_1 = end_time_1 - start_time_1 #to record the total time taken by the user to answer the 5 questions in level 1
    
    html_content_2 = HTML('<p style="font-size: 20px;">Level 2</p>')
    
    #create output widget
    out3 = Output()
    out4 = Output() 

    #display output
    display(out3)
    display(out4)
     
    with out3: display(html_content_2, grid_1)

    #displaying time remaining and numbers in a loop
    for i in range(12, -1, -1): #countdown from 12 to 0
        out4.clear_output(wait=True)
        with out4: display(HTML(f"<h3>Time remaining: {i} seconds</h3>"))
            
        time.sleep(1) 
        
    clear_output(wait=False) #clear the output once time is up.

    start_time_2 = time.time()
            
    global total_score_2 
    total_score_2 = 0
    
    for i in range(len_ques):
        if i == 4:
            display(q5)

        ques = question_bank_1[i]
        ans = answer_1[i]
        answer=input(ques)
        
        if answer == ans:
            print ("Correct!")
            total_score_2 = total_score_2 + 1
        else:
            print(f"Sorry, the correct answer is: {ans}")

        time.sleep(1)
        clear_output(wait=False)
   
    end_time_2=time.time() 
    time_taken_2 = end_time_2 - start_time_2

    html_content_3 = HTML('<p style="font-size: 20px;">Level 3</p>')
    
    #create output widget
    out5 = Output()
    out6 = Output() 

    #display output
    display(out5)
    display(out6)
     
    with out5: display(html_content_3, grid_3)

    #displaying time remaining and numbers in a loop
    for i in range(15, -1, -1): #countdown from 15 to 0
        out6.clear_output(wait=True)
        with out6: display(HTML(f"<h3>Time remaining: {i} seconds</h3>"))
            
        time.sleep(1)
        
    clear_output(wait=False)

    start_time_3 = time.time()
            
    global total_score_3
    total_score_3 = 0
    
    for i in range(len_ques):
        if i == 4:
            display(q5)

        ques = question_bank_3[i]
        ans = answer_3[i]
        answer=input(ques)
        
        if answer == ans:
            print ("Correct!")
            total_score_3 = total_score_3 + 1
        else:
            print(f"Sorry, the correct answer is: {ans}")

        time.sleep(1)
        clear_output(wait=False)

    end_time_3=time.time() 
    time_taken_3 = end_time_3 - start_time_3

    global total_time_taken, total_score
    total_time_taken = time_taken_1 + time_taken_2 + time_taken_3 #sum up the total time taken to complete all 3 levels
    return_total_time_taken = print(f"You took {total_time_taken}s to answers 15 questions")
    
    total_score = total_score_1 + total_score_2 + total_score_3 #sum up the total score scored by user
    return_total_score = print(f"Your total score for the level 1,2 and 3 is {total_score}")
    
   #data collection consent with buttons
        
    print("Please read:")
    print("")
    print("we wish to record your response data")
    print("to an anonymised public data repository. ")
    print("Your data will be used for educational teaching purposes")
    print("practising data analysis and visualisation.")
    print("")
    print("Please click yes in the button below if you consent to the upload.")
    
    btn_yes = widgets.Button(description="Yes")
    btn_no = widgets.Button(description="No")
    
    btn_yes.on_click(register_btn_event)
    btn_no.on_click(register_btn_event)

    
    panel = widgets.HBox([btn_yes, btn_no])
    display(panel)

    event = wait_for_event()
    if event['description'] == "Yes":
        on_button_click()  # Function to handle "Yes" button click
    elif event['description'] == "No":
        register_event()  # Function to handle "No" button click
        
    return return_total_time_taken, return_total_score

#judging matrix: speed and accuracy
run_memory_test()

You took 39.73823523521423s to answers 15 questions
Your total score for the level 1,2 and 3 is 3
Please read:

we wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please click yes in the button below if you consent to the upload.


Thanks - your data will be uploaded.


(None, None)